In [ ]:
from fitizens_libraries.load_data import load_labeled_data
from scipy.signal import find_peaks
from custom_libraries.load_data_for_label import load_data_for_label
from custom_libraries.merge_data import merge_data
import pandas as pd
import os
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
import joblib 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
import numpy as np
from custom_libraries.find_peaks import find_peaks_cust
from sklearn.preprocessing import StandardScaler

In [ ]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True) #Referenciamos la carpeta LABELED en la que están las carpetas zip con los json
#Ahora voy a iterar en esa carpeta LABELED para obtener la ruta de los archivos, que es el LABELED/NOMBRE y eso lo guardo en una lista
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]

In [ ]:
datos = load_data_for_label(filelist=file_names[0:2],
                       signals= signals)

In [ ]:
datos

In [ ]:
len(datos)

In [ ]:
type(datos)

In [ ]:
datos[0]

In [ ]:
df = pd.concat(datos,axis=0)

In [ ]:
df

In [ ]:
prim_1000 = df.head(4000)
fig = px.line(prim_1000, x=prim_1000.index, y='linAccZ', title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
import plotly.graph_objects as go

# Crear la figura de la serie de tiempo
fig = go.Figure()

# Añadir la línea de la serie de tiempo
fig.add_trace(go.Scatter(x=prim_1000.index, y=prim_1000['linAccZ'], mode='lines', name='linAccZ'))

# Filtrar los índices de los picos
peaks_indices = prim_1000.index[prim_1000['peaks'].notna()]

# Añadir marcadores para los picos en la gráfica
fig.add_trace(go.Scatter(x=peaks_indices, y=prim_1000.loc[peaks_indices, 'linAccZ'],
                         mode='markers', marker=dict(color='red'), name='peaks'))

# Establecer el título de la gráfica
fig.update_layout(title='Time serie of exercise linAccZ')

# Mostrar la gráfica
fig.show()

In [ ]:
df.head()

In [ ]:
############ ESTA ES PARA EL SQUAT EN EL SEGUNDO BONCHE DE DATOS #################
############## NO LO EJECUTES #####################
nuevo_df_predicciones = pd.DataFrame()
modelo = joblib.load('modeloXGB.pkl')
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
            'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
            'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
            'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
            'median_gyroX_mod', 'median_linAccX_mod']

nuevo_df_list = []  # Lista para almacenar las filas

for idx, fila in df.iloc[51:].iterrows():
    warnings.filterwarnings('ignore')
    nuevo_df_list.append(fila.copy())  # Añadir la fila a la lista
    if pd.notna(fila['peaks']):
        nuevo_df = pd.DataFrame(nuevo_df_list)  # Convertir la lista en DataFrame
        
        promedio = nuevo_df.mean().add_prefix('mean_')
        std = nuevo_df.std().add_prefix('std_')
        median = nuevo_df.median().add_prefix('median_')
        nuevo_df_estad = pd.DataFrame().append(pd.concat([promedio, std, median]), ignore_index=True)
        
        X1 = nuevo_df_estad[features].copy()
        y_pred = modelo.predict(X1)
        
        inicio_ventana = nuevo_df.index[0] 
        fin_ventana = nuevo_df.index[-1]
        
        X1['start'] = inicio_ventana
        X1['end'] = fin_ventana
        X1['prediccion'] = y_pred[0]
    
        nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)
        break

In [ ]:
nuevo_df_predicciones

## Fraccionar los datos segun los picos

## XGBOOST

In [ ]:
nuevo_df_predicciones = pd.DataFrame()
nuevo_df_list = []  # Lista para almacenar las filas
modelo = joblib.load('modeloXGB.pkl')
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
            'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
            'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
            'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
            'median_gyroX_mod', 'median_linAccX_mod']

for idx, fila in df.iterrows():
    warnings.filterwarnings('ignore')
    nuevo_df_list.append(fila.copy())  # Añadir la fila a la lista
    
    if pd.notna(fila['peaks']):
        nuevo_df = pd.DataFrame(nuevo_df_list)  # Convertir la lista en DataFrame

        promedio = nuevo_df.mean().add_prefix('mean_')
        std = nuevo_df.std().add_prefix('std_')
        median = nuevo_df.median().add_prefix('median_')
        nuevo_df_estad = pd.DataFrame().append(pd.concat([promedio, std, median]), ignore_index=True)

        X1 = nuevo_df_estad[features].copy()
        y_pred = modelo.predict(X1)


        X1['start'] = nuevo_df.index[0]
        X1['end'] = nuevo_df.index[-1]
        X1['prediccion'] = y_pred[0]

        nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

        # Limpiar la lista para la próxima ventana
        nuevo_df_list = [fila.copy()]

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 1][['start', 'end', 'prediccion']]
squats_detection

## Naive Bayes

In [ ]:
nuevo_df_predicciones = pd.DataFrame()
nuevo_df_list = []  # Lista para almacenar las filas
modelo2 = joblib.load('modeloNB.pkl')
scaler =  joblib.load('scaler3.pkl')

features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
            'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
            'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
            'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
            'median_gyroX_mod', 'median_linAccX_mod']

for idx, fila in df.iterrows():
    warnings.filterwarnings('ignore')
    nuevo_df_list.append(fila.copy())  # Añadir la fila a la lista
    
    if pd.notna(fila['peaks']):
        nuevo_df = pd.DataFrame(nuevo_df_list)  # Convertir la lista en DataFrame

        promedio = nuevo_df.mean().add_prefix('mean_')
        std = nuevo_df.std().add_prefix('std_')
        median = nuevo_df.median().add_prefix('median_')
        nuevo_df_estad = pd.DataFrame().append(pd.concat([promedio, std, median]), ignore_index=True)

        X1 = nuevo_df_estad[features].copy()
        X1_full = scaler.transform(X1)
        y_pred = modelo2.predict(X1_full)

        X1['start'] = nuevo_df.index[0]
        X1['end'] = nuevo_df.index[-1]
        X1['prediccion'] = y_pred[0]
        
        nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

        # Limpiar la lista para la próxima ventana
        nuevo_df_list = [fila.copy()]

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 1][['start', 'end', 'prediccion']]
squats_detection

## Regresion Logistica

In [ ]:
nuevo_df_predicciones = pd.DataFrame()
nuevo_df_list = []  # Lista para almacenar las filas
modelo3 = joblib.load('modeloLR.pkl')
scaler =  joblib.load('scaler3.pkl')
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
            'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
            'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
            'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
            'median_gyroX_mod', 'median_linAccX_mod']

for idx, fila in df.iterrows():
    warnings.filterwarnings('ignore')
    nuevo_df_list.append(fila.copy())  # Añadir la fila a la lista
    
    if pd.notna(fila['peaks']):
        nuevo_df = pd.DataFrame(nuevo_df_list)  # Convertir la lista en DataFrame

        promedio = nuevo_df.mean().add_prefix('mean_')
        std = nuevo_df.std().add_prefix('std_')
        median = nuevo_df.median().add_prefix('median_')
        nuevo_df_estad = pd.DataFrame().append(pd.concat([promedio, std, median]), ignore_index=True)

        X1 = nuevo_df_estad[features].copy()
        X1_full = scaler.transform(X1)
        y_pred = modelo3.predict(X1_full)
        
        X1['start'] = nuevo_df.index[0]
        X1['end'] = nuevo_df.index[-1]
        X1['prediccion'] = y_pred[0]

        nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

        # Limpiar la lista para la próxima ventana
        nuevo_df_list = [fila.copy()]

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 1][['start', 'end', 'prediccion']]
squats_detection

# Hacia abajo NO

In [ ]:
def extract_squat_time_range(df_workout):
    # Identify start and end times
    squats_detection = pd.concat([df_workout[ df_workout.first_sample == True ], df_workout[ df_workout.last_sample == True ]]).sort_index()
    
    start_times = squats_detection[squats_detection['first_sample']].index
    end_times = squats_detection[squats_detection['last_sample']].index
    
    paired_times = list(zip(start_times, end_times))
    
    # Create new DataFrame
    return pd.DataFrame(paired_times, columns=['start', 'end'])

In [ ]:
extract_squat_time_range(df)

In [ ]:
len(df)

In [ ]:
import numpy as np
np.unique(df.exercise,return_counts=True)

In [ ]:
import seaborn as sns
sns.countplot(x=df['exercise'], label = "squat")

In [ ]:
df.columns

In [ ]:
df.loc[df['exercise'] == 'NO_EXERCISE','exercising_periods'].unique()

In [ ]:
import plotly.express as px
prim_1000 = df.head(2000)
fig = px.line(prim_1000, x=prim_1000.index, y='linAccZ', color="exercise",title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
prim_100 = df.head(167)
fig = px.line(prim_100, x=prim_100.index, y='linAccZ', color="exercise",title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
siguientes_170 = df.iloc[168:336]

fig = px.line(siguientes_170, x=siguientes_170.index, y='linAccZ', color="exercise", title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
siguientes_170 = df.iloc[336:510]

fig = px.line(siguientes_170, x=siguientes_170.index, y='linAccZ', color="exercise", title='Time serie of exercise linAccZ')
fig.show()

In [ ]:
modelo = joblib.load('modeloXGB.pkl')

In [ ]:
scaler =  joblib.load('scaler3.pkl')

In [ ]:
scaler

In [ ]:
X = df.drop(['peaks', 'first_sample','last_sample', 'first_sample_closest_peak', 
             'last_sample_closest_peak','exercise','accY', 'exercising_periods'], axis=1)
y = df['exercise']

In [ ]:
X.columns

In [ ]:
y

## Modelo XGBOOST

In [ ]:
tamano_ventana = 170 
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
       'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
       'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
       'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
       'median_gyroX_mod', 'median_linAccX_mod']

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    # Seleccionar la ventana de datos
    ventana = X.iloc[i:i + tamano_ventana]
    #print(ventana)

    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    nuevo_df = pd.DataFrame().append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)

    
    X1= nuevo_df[features].copy()
    y_pred = modelo.predict(X1)
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 1][['start', 'end', 'prediccion']]
squats_detection

In [ ]:
np.unique(y_pred, return_counts=True)

## Modelo de Naive Bayes

In [ ]:
tamano_ventana = 170
nuevo_df = pd.DataFrame
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
       'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
       'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
       'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
       'median_gyroX_mod', 'median_linAccX_mod']

modelo2 = joblib.load('modeloNB.pkl')

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    # Seleccionar la ventana de datos
    ventana = X.iloc[i:i + tamano_ventana]
    #print(ventana)

    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    nuevo_df = pd.DataFrame().append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)

 
    X1= nuevo_df[features].copy()
    X1_full = scaler.transform(X1)
    y_pred = modelo2.predict(X1_full)
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 1][['start', 'end', 'prediccion']]
squats_detection

## Modelo Regresion logistica

In [ ]:
tamano_ventana = 170 
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
       'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
       'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
       'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
       'median_gyroX_mod', 'median_linAccX_mod']
# Scale Data
modelo3 = joblib.load('modeloLR.pkl')

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    # Seleccionar la ventana de datos
    ventana = X.iloc[i:i + tamano_ventana]
    #print(ventana)

    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    nuevo_df = pd.DataFrame().append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)
    #nuevo_df = nuevo_df.append(pd.concat([promedio_serie.add_prefix('mean_'), std_serie.add_prefix('std_'), median_serie.add_prefix('median_')]), ignore_index=True)

    
    X1= nuevo_df[features].copy()
    X1_full = scaler.transform(X1)
    y_pred = modelo3.predict(X1_full)
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 1][['start', 'end', 'prediccion']]
squats_detection

## Ventana por tiempo

Teniendo en cuenta las graficas, ahora se porbará hacer ventanas con tiempos

## XGBOOST

In [ ]:
from datetime import timedelta
X.index = pd.to_datetime(X.index)
tamano_ventana_segundos = 1.85  # Ahora la ventana es de 1.85 segundos
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
            'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
            'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
            'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
            'median_gyroX_mod', 'median_linAccX_mod']

# Ajusta la ventana inicial para incluir el primer dato
ventana_inicio = X.index[0]
ventana_fin = ventana_inicio + timedelta(seconds=tamano_ventana_segundos)

while ventana_fin <= X.index[-1]:
    # Seleccionar la ventana de datos basada en el tiempo
    ventana = X.loc[(X.index >= ventana_inicio) & (X.index <= ventana_fin)]
    
    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    
    nuevo_df = pd.DataFrame().append(pd.concat([promedio_serie.add_prefix('mean_'), 
                                           std_serie.add_prefix('std_'), 
                                           median_serie.add_prefix('median_')]), ignore_index=True)

    X1 = nuevo_df[features].copy()
    y_pred = modelo.predict(X1)
    
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

    # Mover la ventana al siguiente intervalo de tiempo
    ventana_inicio = ventana_fin
    ventana_fin = ventana_inicio + timedelta(seconds=tamano_ventana_segundos)

In [ ]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 1][['start', 'end', 'prediccion']]
squats_detection

In [ ]:
np.unique(y_pred,return_counts=True)

## Naive Bayes

In [ ]:
from datetime import timedelta

X.index = pd.to_datetime(X.index)
modelo2 = joblib.load('modeloNB.pkl')

tamano_ventana_segundos = 1.85  # Ahora la ventana es de 1.85 segundos
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
            'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
            'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
            'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
            'median_gyroX_mod', 'median_linAccX_mod']

# Ajusta la ventana inicial para incluir el primer dato
ventana_inicio = X.index[0]
ventana_fin = ventana_inicio + timedelta(seconds=tamano_ventana_segundos)

while ventana_fin <= X.index[-1]:
    # Seleccionar la ventana de datos basada en el tiempo
    ventana = X.loc[(X.index >= ventana_inicio) & (X.index <= ventana_fin)]
    
    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    
    nuevo_df = pd.DataFrame().append(pd.concat([promedio_serie.add_prefix('mean_'), 
                                           std_serie.add_prefix('std_'), 
                                           median_serie.add_prefix('median_')]), ignore_index=True)

    X1 = nuevo_df[features].copy()
    X1_full = scaler.transform(X1)
    y_pred = modelo2.predict(X1)
    
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

    # Mover la ventana al siguiente intervalo de tiempo
    ventana_inicio = ventana_fin
    ventana_fin = ventana_inicio + timedelta(seconds=tamano_ventana_segundos)


In [ ]:
nuevo_df_predicciones.head()

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

## Regresion logistica

In [ ]:
from datetime import timedelta

X.index = pd.to_datetime(X.index)
modelo3 = joblib.load('modeloLR.pkl')

tamano_ventana_segundos = 1.85  # Ahora la ventana es de 1.85 segundos
nuevo_df_predicciones = pd.DataFrame()
features = ['mean_accX', 'mean_accZ', 'mean_linAccZ', 'mean_accZ_mod',
            'mean_gyroX_mod', 'mean_linAccX_mod', 'std_accZ', 'std_gyroX', 'std_gyroZ',
            'std_magnX', 'std_linAccX', 'std_accZ_mod', 'std_linAccX_mod',
            'median_accX', 'median_accZ', 'median_linAccZ', 'median_accZ_mod',
            'median_gyroX_mod', 'median_linAccX_mod']

# Ajusta la ventana inicial para incluir el primer dato
ventana_inicio = X.index[0]
ventana_fin = ventana_inicio + timedelta(seconds=tamano_ventana_segundos)

while ventana_fin <= X.index[-1]:
    # Seleccionar la ventana de datos basada en el tiempo
    ventana = X.loc[(X.index >= ventana_inicio) & (X.index <= ventana_fin)]
    
    # Calcular estadísticas (promedio, desviación estándar, mediana) para cada columna de la ventana
    promedio_serie = ventana.mean()
    std_serie = ventana.std()
    median_serie = ventana.median()
    
    nuevo_df = pd.DataFrame().append(pd.concat([promedio_serie.add_prefix('mean_'), 
                                           std_serie.add_prefix('std_'), 
                                           median_serie.add_prefix('median_')]), ignore_index=True)

    X1 = nuevo_df[features].copy()
    X1_full = scaler.transform(X1)
    y_pred = modelo3.predict(X1)
    
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred[0]
    
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

    # Mover la ventana al siguiente intervalo de tiempo
    ventana_inicio = ventana_fin
    ventana_fin = ventana_inicio + timedelta(seconds=tamano_ventana_segundos)

In [ ]:
nuevo_df_predicciones.head()

In [ ]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)